In [21]:
import pandas as pd
import numpy as np


book = pd.read_excel('test.xlsx', sheet_name='sheet1')

Реакция на колличество аккаунтов, привязанных к одному Customer ID

In [22]:
cb_1 = book.groupby(['Customer ID'])['Account Number'].nunique()
cb_1 = cb_1.reset_index()

conditions_1 = [
    (cb_1['Account Number'] < 2),
    (cb_1['Account Number'] == 2),
    (cb_1['Account Number'] == 3),
    (cb_1['Account Number'] == 4),
    (cb_1['Account Number'] > 4),
]

values_1 = [0, 10, 20, 30, 50]

cb_1['estimation_1'] = np.select(conditions_1, values_1)

Реакция на колличество попыток оплат на уникальные суммы в сутки с одного Customer ID

In [23]:
cb_2 = book.groupby(['Customer ID', 'Operation created at Date'])['Channel Amount'].nunique()
cb_2 = cb_2.reset_index()
cb_2 = cb_2.groupby(['Customer ID'])['Channel Amount'].sum()
cb_2 = cb_2.reset_index()

conditions_2 = [
    (cb_2['Channel Amount'] < 3),
    (cb_2['Channel Amount'] < 6),
    (cb_2['Channel Amount'] < 9),
    (cb_2['Channel Amount'] < 11),
    (cb_2['Channel Amount'] >= 11),
]

values_2 = [0, 10, 20, 30, 50]

cb_2['estimation_2'] = np.select(conditions_2, values_2)

Реакция на колличество Email, привязанных к Customer ID

In [24]:
cb_3 = book.groupby(['Customer ID'])['Customer IP V4 V6'].nunique()
cb_3 = cb_3.reset_index()

conditions_3 = [
    (cb_3['Customer IP V4 V6'] < 3),
    (cb_3['Customer IP V4 V6'] < 4),
    (cb_3['Customer IP V4 V6'] < 5),
    (cb_3['Customer IP V4 V6'] >= 5),
]

values_3 = [0, 10, 20, 50]

cb_3['estimation_3'] = np.select(conditions_3, values_3)

Реакция на операции, в которых замечено несоответствие страны карты эмитента со страной по IP относительно Customer ID

In [25]:
cb_4 = book
cb_4['Compare'] = cb_4['Issuer Country Name'] != cb_4['Country By IP Name']
cb_4 = cb_4[['Customer ID', 'Issuer Country Name', 'Country By IP Name', 'Compare']]
cb_4 = cb_4.groupby(['Customer ID'])['Compare'].sum()
cb_4 = cb_4.reset_index()
conditions_4 = [
    (cb_4['Compare'] < 4),
    (cb_4['Compare'] < 6),
    (cb_4['Compare'] < 8),
    (cb_4['Compare'] >= 8),
]

values_4 = [0, 20, 30, 50]

cb_4['estimation_4'] = np.select(conditions_4, values_4)

Подведение финальных расчетов и вывод их в таблицу

In [26]:
cb_a = pd.merge(cb_1, cb_2)
cb_b = pd.merge(cb_3, cb_4)
cb = pd.merge(cb_a, cb_b)
cb['Full Estimation'] = cb[['estimation_1','estimation_2','estimation_3','estimation_4']].sum(axis=1)
cb = cb.loc[cb['Full Estimation'] > 50]
cb = cb.sort_values("Full Estimation", ascending=False)
cb = cb.sort_values("Full Estimation", ascending=False)
cb_1 = cb.loc[(50 < cb['Full Estimation']) & (cb['Full Estimation'] < 100)]
cb_2 = cb.loc[100 < cb['Full Estimation']]

with pd.ExcelWriter("finall_output_1.xlsx") as writer:
    cb_1.to_excel(writer, sheet_name="Suspicion", index=False)
    cb_2.to_excel(writer, sheet_name="Fraud", index=False)